드라이브 마운트 (20초)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


zip 파일 코렙으로 복사 (10분) - train 파일만 복사하기

In [ ]:
!cp "/content/drive/MyDrive/OD_project/raw/차량상태인식 (1).zip" /content

zip 압축 해제 (6분)

In [ ]:
!unzip -q "/content/차량상태인식 (1).zip" -d /content/my_dataset

train/val/test로 분할

In [ ]:
import os
import glob
import cv2
import shutil
import pandas as pd


def prepare_yolo_dataset_by_csv(root_path, csv_path, output_path):
    df = pd.read_csv(csv_path, encoding='utf-8-sig')
    df = df[['env', 'split']].dropna()
    df = df[df['split'].isin(['train', 'val', 'test'])]

    split_map = dict(zip(df['env'], df['split']))
    print(f"✅ CSV 로드 완료: {len(split_map)}개의 환경(env) 설정 읽음.")

    image_paths = glob.glob(os.path.join(root_path, "**/img/*.png"), recursive=True)
    print(f"총 {len(image_paths)}개의 이미지를 찾았습니다.")

    if len(image_paths) == 0:
        print("❌ 이미지를 찾지 못했습니다. root_path 확인 필요")
        return

    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(output_path, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(output_path, split, 'labels'), exist_ok=True)

    processed_count = 0
    missing_env_count = 0

    for img_path in image_paths:
        folder_name = img_path.split(os.sep)[-3]
        file_name = os.path.basename(img_path)

        target_split = split_map.get(folder_name)
        if not target_split:
            missing_env_count += 1
            continue

        new_base = f"{folder_name}_{file_name}"
        new_label = new_base.replace(".png", ".txt")

        org_label_path = img_path.replace("img", "new_txt").replace(".png", ".txt")
        if not os.path.exists(org_label_path):
            continue

        img = cv2.imread(img_path)
        if img is None:
            continue
        h, w, _ = img.shape

        yolo_labels = []
        with open(org_label_path) as f:
            for line in f:
                data = list(map(float, line.split()))
                if len(data) < 5:
                    continue

                x1, y1, x2, y2 = data[:4]
                class_id = int(data[4])

                xc = ((x1 + x2) / 2) / w
                yc = ((y1 + y2) / 2) / h
                nw = (x2 - x1) / w
                nh = (y2 - y1) / h

                yolo_labels.append(
                    f"{class_id} {xc:.6f} {yc:.6f} {nw:.6f} {nh:.6f}"
                )

        if yolo_labels:
            shutil.copy(
                img_path,
                os.path.join(output_path, target_split, "images", new_base)
            )
            with open(
                os.path.join(output_path, target_split, "labels", new_label), "w"
            ) as f:
                f.write("\n".join(yolo_labels))

            processed_count += 1

    print("-" * 30)
    print("✨ 처리 완료!")
    print(f"- 성공적으로 변환된 이미지: {processed_count}장")
    print(f"- CSV에 정의되지 않아 제외된 폴더: {missing_env_count}개")
    print(f"- 결과 저장소: {os.path.abspath(output_path)}")

분할 실행 (10분)

In [ ]:
prepare_yolo_dataset_by_csv(
    root_path="/content/my_dataset",
    csv_path="/content/drive/MyDrive/OD_project/split_assignment.csv",
    output_path="/content/my_car_dataset"
)

✅ CSV 로드 완료: 165개의 환경(env) 설정 읽음.
총 42096개의 이미지를 찾았습니다.
------------------------------
✨ 처리 완료!
- 성공적으로 변환된 이미지: 33187장
- CSV에 정의되지 않아 제외된 폴더: 8909개
- 결과 저장소: /content/my_car_dataset


data.yaml 생성

In [ ]:
import yaml
import os

# 데이터셋 루트
DATASET_ROOT = '/content/my_car_dataset'

# 클래스 수와 이름 미리 알고 있을 때
nc = 2
names = ['car', 'bus']

# data.yaml 딕셔너리
data_yaml = {
    'path': DATASET_ROOT,
    'train': 'train/images',
    'val': 'val/images',
    'nc': nc,
    'names': names
}

# Drive 경로에 저장
yaml_path = os.path.join(DATASET_ROOT, 'data.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, sort_keys=False)

print(f"✅ data.yaml 생성 완료: {yaml_path}")

✅ data.yaml 생성 완료: /content/my_car_dataset/data.yaml


데이터셋 확인

In [ ]:
!ls /content/my_car_dataset

data.yaml  test  train	val


ultralytics 다운로드

In [ ]:
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.9 MB/s eta 0:00:00


학습

In [ ]:
from ultralytics import YOLO
import os

# 1. 모델 로드 (YOLOv11n - 가장 가벼운 모델)
# 이전 실험(m)과 비교하기 위해 n 모델을 사용합니다.
model = YOLO('yolo11n.pt')

# 2. 데이터셋 경로
DATA_YAML_PATH = '/content/my_car_dataset/data.yaml'

# 3. 기본 학습 설정
results = model.train(
    data=DATA_YAML_PATH,
    epochs=100,

    # --- 시스템 설정 ---
    device=0,
    project='/content/my_car_dataset',
    name='yolo11n_v1',
    amp=True,
    pretrained=True,
    exist_ok=True,
)

print("🚀 YOLOv11n 기본 학습이 완료되었습니다!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.252 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=0, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/my_car_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=F

test 예측하기

In [ ]:
from ultralytics import YOLO

model = YOLO(
    '/content/runs/detect/weights/best.pt'
)

metrics = model.val(
    data='/content/drive/MyDrive/OD_project/data.yaml',
    split='test',      # 🔥 핵심
    imgsz=640,
    batch=16,
    device=0
)


이미지들을 mp4 영상으로 만들기

In [ ]:
import os
import cv2

img_dir = "/content/runs/detect/predict2"   # ← 실제 경로로 맞춰줘
output_video = "/content/predict2_result.mp4"
fps = 20   # 영상 속도 (프레임/초)

images = sorted([
    f for f in os.listdir(img_dir)
    if f.lower().endswith(('.jpg', '.png'))
])

assert len(images) > 0, "❌ 이미지가 없습니다."

# 첫 이미지로 해상도 설정
first = cv2.imread(os.path.join(img_dir, images[0]))
h, w, _ = first.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(output_video, fourcc, fps, (w, h))

for img_name in images:
    frame = cv2.imread(os.path.join(img_dir, img_name))
    video.write(frame)

video.release()

print("✅ 영상 생성 완료:", output_video)
